Some relevant git directories:
https://github.com/FarizAfkar/TA/blob/master/gui.py
https://github.com/wblgers/hmm_speech_recognition_demo/blob/master/demo.py
https://github.com/JarbasAl/easy_gmmhmm/blob/master/easy_gmmhmm/train.py

In [1]:
from google.colab import drive

# Mount folder
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ls "/content/drive/My Drive/Correlaid/"

AdditionalUtterances.zip  datadic_label_mfcc.pkl  trial_gm_hmm.ipynb
audio_files		  SampleSubmission.csv
audio_files.zip		  Train.csv


In [3]:
# Set working directory
%cd "/content/drive/My Drive/Correlaid/"

/content/drive/My Drive/Correlaid


In [4]:
wd = "/content/drive/My Drive/Correlaid/"

In [5]:
!ls 

AdditionalUtterances.zip  datadic_label_mfcc.pkl  trial_gm_hmm.ipynb
audio_files		  SampleSubmission.csv
audio_files.zip		  Train.csv


In [6]:
#!unzip -q audio_files.zip
!unzip -q AdditionalUtterances.zip

In [7]:
!ls

AdditionalUtterances.zip  datadic_label_mfcc.pkl  Train.csv
audio_files		  latest_keywords	  trial_gm_hmm.ipynb
audio_files.zip		  SampleSubmission.csv


In [9]:
!pip install hmmlearn
!pip install python_speech_features

     |████████████████████████████████| 368kB 2.8MB/s 
  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5887 sha256=4b7bed2e74f3d05f62f488a72409ea6bcde6ee63becef47935d5071b854e26be
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [10]:
import librosa
import python_speech_features as psf
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from hmmlearn import hmm
import os
from sklearn.model_selection import train_test_split
import pickle
#from scipy.io import wavfile

In [30]:
!ls

AdditionalUtterances.zip  datadic_label_mfcc.pkl  Train.csv
audio_files		  latest_keywords	  trial_gm_hmm.ipynb
audio_files.zip		  SampleSubmission.csv


In [78]:
#import sys
#sys.path.append('/content/drive/My Drive/Colab Notebooks')
import autoreload
%reload_ext autoreload
%autoreload 2

In [125]:
#!pip install import-ipynb
import import_ipynb
import importlib
#!mv Utils.ipynb wd
import importlib
import Utils

importlib.reload(Utils)

importing Jupyter notebook from /nbs/Utils.ipynb


<module 'Utils' from '/nbs/Utils.ipynb'>

In [91]:
#dir(Utils)

In [32]:
!pip install kora -q
from kora import drive
drive.link_nbs()


     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 51kB 4.4MB/s 


# MFCC

Each audio files have different lengths. Using librosa I have used a fixed length of signal. On which mfcc is done. Number of mfcc used is 39, which is standard.

Shape of output is (39,108), where 108 is nsamples = duration x sample_rate/hop_size. As I have fixed duration x sample_rate to be 55260, the shape makes sense.

With varying shape from audio signals, it can't be used in gmmhmm as the np.vstack method fails for varying array structure.

In [18]:


def get_mfcc(full_audio_path):
    '''
    https://stackoverflow.com/questions/37963042/python-librosa-what-is-the-default-frame-size-used-to-compute-the-mfcc-feature
    '''
    #audio_length_seconds = 2.5
    sample_rate = 44100
    hop_size = 512#347*audio_length_seconds
    fmin = 20
    fmax = sample_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    #samples = sample_rate * audio_length_seconds

    wave, __ =  librosa.load(full_audio_path)
    #clip on fix length
    wave = librosa.util.fix_length(wave, 55260, mode='constant')
    
    mfcc_features = librosa.feature.mfcc(y=wave, sr=sample_rate, n_mfcc =39,hop_length=int(hop_size),n_fft=n_fft,fmin=fmin,fmax=fmax)#n_mels=n_mels,
    #normalize
    mfcc_features -= (np.mean(mfcc_features, axis=0) + 1e-8)
    
    
    return mfcc_features


# Data structuring

We have 193 labels and around 3-12 samples for each label. We here created a dictionary with key as label. For each key, a list of mfcc features is created where each item is an mfcc.

Later we divided the dictionary into test and training dictionary where each label has [0.8:0.2] in [train:test].

In [16]:
train = pd.read_csv('Train.csv')
train.head()
#train = train.iloc[:5]
#train

,fn,label
0,audio_files/IV38R7F.wav,akawuka
1,audio_files/KM4SKWT.wav,banana
2,audio_files/F5POSU9.wav,obulwadde
3,audio_files/MMVDXG2.wav,nnyaanya
4,audio_files/9TVM96F.wav,pampu


In [123]:
#create a dictionary of keys as label and list of corresponsinds file paths as values
startfresh = True
if startfresh:
  featuredic = {}
  datadic = {}
  #add old file paths to the dic
  train.apply(lambda x: Utils.createFilepathDic(x,datadic,wd),axis=1)
  #add new files to the dic
  newfile = 'latest_keywords/'
  datadic = Utils.add_to_DataDic(datadic,newfile,wd)
else:
  pass


['ebyobulimi', 'ebijjanjalo', 'obulimi', 'doodo', 'munyeera', 'akatungulu', 'pampu', 'ennimiro', 'ebibala', 'okusaasaanya', 'mpeke', 'sikungula', 'ebirime', 'okulima', 'ebinyebwa', 'emboga', 'ebisaanyi', 'ebiwojjolo', 'omuyembe', 'ebijanjaalo', 'muceere', 'amatooke', 'eppapaali', 'ekisaanyi', 'ekigimusa', 'ekyeya', 'amakoola', 'okusiga', 'emmwanyi', 'lumonde', 'omulimisa', 'ekirime', 'ekitooke', 'ebikolo', 'matooke', 'mucungwa', 'obusaanyi', 'ennyaanya', 'okufuuyira', 'ebisoolisooli', 'ekibala', 'miceere', 'okusimba', 'ebikoola', 'akammwanyi', 'obutungulu', 'ennima', 'okukkoola', 'ekikajjo', 'okugimusa', 'ekirwadde', 'ebigimusa', 'nambaale', 'eddagala', 'ebiwuka', 'ensigo', 'emiyembe', 'ebbugga', 'kassooli', 'ebitooke', 'ekimera', 'kisaanyi', 'omusiri', 'namuginga', 'kaamulali', 'nnyaanya', 'endokwa', 'emisiri', 'ensuku', 'nakavundira', 'amalagala', 'bulimi', 'ndwadde', 'kikolo', 'obuwuka', 'ekiwojjolo', 'ekikoola', 'akawuka', 'bibala', 'endagala', 'nnimiro', 'ejjobyo', 'kasaanyi', 'nn

In [133]:
#get number of files for each
for label in datadic.keys():
    print(label,len(datadic[label]))

akawuka 29
banana 4
obulwadde 27
nnyaanya 20
pampu 13
obutunda 20
plantation 8
ensujju 20
okulimibwa 14
mpeke 17
okusaasaana 29
ebigimusa 31
ekikolo 13
farm 7
kisaanyi 26
kikajjo 18
ekisaanyi 29
ndwadde 31
omusiri 13
butterfly 3
munyeera 29
eggobe 19
ebiwojjolo 27
ebisoolisooli 22
namuginga 29
okugimusa 30
maize streak virus 4
ekirime 15
miceere 18
sikungula 16
lumonde 20
okukungula 15
cassava 3
ebirime 13
ebijanjaalo 20
weeding 8
garden 7
drought 6
leaves 7
insect 5
akatungulu 18
seed 7
pepper 4
matooke seedlings 3
harvesting 7
medicine 4
nursery bed 7
mucungwa 19
endwadde 31
pawpaw 4
enkota 19
ensiringanyi 30
kassooli 25
okufuuyira 31
caterpillars 4
ekijanjaalo 18
okukkoola 14
crop 8
okulima 14
endagala 14
kaamulali 20
ennima 14
omuceere 19
micungwa 22
ebisaanyi 29
plant 7
eddagala 30
ennimiro 13
amakoola 16
ebiwuka 32
ekigimusa 24
bibala 19
beans 3
nnimiro 14
ebinyebwa 20
passion fruit 3
Spinach 4
okuzifuuyira 27
ekirwadde 24
nakavundira 27
nfukirira 15
onion 3
ddagala 30
muwogo 21


In [128]:
FeatureObj = Utils.GetFeatureDic(Utils.get_mfcc)

mfccdic = FeatureObj.createFeatureDic(datadic)

In [129]:
#pickle.dump(mfccdic, open("datadic_label_mfcc.pkl", "wb"))
#mfccdic = pickle.load(open("datadic_label_mfcc.pkl", "rb"))

In [131]:
mfccdic['banana']

[array([[-157.41505861, -140.62549653, -131.98293745, ..., -103.59628819,
          -98.56952958,  -96.68177408],
        [ 126.49446541,  131.88996123,  132.04890086, ...,  139.06540462,
          142.20352187,  140.23256196],
        [ -10.35579453,   -7.02519165,   -9.42981921, ...,  -28.053868  ,
          -23.71029414,  -19.76692737],
        ...,
        [   0.18143929,   -4.45917876,   -9.33368556, ...,  -10.04441696,
           -7.66679029,   -5.2932107 ],
        [   7.56248586,    3.10596177,   -0.18901496, ...,    3.34361729,
            4.2003996 ,    4.58129572],
        [   0.24154321,   -2.36108392,   -1.98257494, ...,   -5.14918538,
           -8.63024009,   -9.74599578]]),
 array([[-104.06876943,  -93.03970234,  -89.44083043, ...,  -86.27650804,
          -88.25720961,  -93.56880685],
        [ 119.22463326,  126.49260331,  131.43240417, ...,  150.77437996,
          148.2742079 ,  144.01530158],
        [ -14.51007393,  -21.12049539,  -25.55901934, ...,  -22.09539879,

In [134]:
#split data into train and test dic
featuredictrain = {}
featuredictest = {}
for label in datadic.keys():
    #print(label)
    featuredictrain[label], featuredictest[label] = train_test_split(mfccdic[label],test_size=0.2)
    print(label,len(featuredictrain[label]),len(featuredictest[label]))

akawuka 23 6
banana 3 1
obulwadde 21 6
nnyaanya 16 4
pampu 10 3
obutunda 16 4
plantation 6 2
ensujju 16 4
okulimibwa 11 3
mpeke 13 4
okusaasaana 23 6
ebigimusa 24 7
ekikolo 10 3
farm 5 2
kisaanyi 20 6
kikajjo 14 4
ekisaanyi 23 6
ndwadde 24 7
omusiri 10 3
butterfly 2 1
munyeera 23 6
eggobe 15 4
ebiwojjolo 21 6
ebisoolisooli 17 5
namuginga 23 6
okugimusa 24 6
maize streak virus 3 1
ekirime 12 3
miceere 14 4
sikungula 12 4
lumonde 16 4
okukungula 12 3
cassava 2 1
ebirime 10 3
ebijanjaalo 16 4
weeding 6 2
garden 5 2
drought 4 2
leaves 5 2
insect 4 1
akatungulu 14 4
seed 5 2
pepper 3 1
matooke seedlings 2 1
harvesting 5 2
medicine 3 1
nursery bed 5 2
mucungwa 15 4
endwadde 24 7
pawpaw 3 1
enkota 15 4
ensiringanyi 24 6
kassooli 20 5
okufuuyira 24 7
caterpillars 3 1
ekijanjaalo 14 4
okukkoola 11 3
crop 6 2
okulima 11 3
endagala 11 3
kaamulali 16 4
ennima 11 3
omuceere 15 4
micungwa 17 5
ebisaanyi 23 6
plant 5 2
eddagala 24 6
ennimiro 10 3
amakoola 12 4
ebiwuka 25 7
ekigimusa 19 5
bibala 15 4


In [49]:
#train.label.value_counts().rename_axis('count').reset_index().head(20)
datadic.keys()

dict_keys(['akawuka', 'banana', 'obulwadde', 'nnyaanya', 'pampu', 'obutunda', 'plantation', 'ensujju', 'okulimibwa', 'mpeke', 'okusaasaana', 'ebigimusa', 'ekikolo', 'farm', 'kisaanyi', 'kikajjo', 'ekisaanyi', 'ndwadde', 'omusiri', 'butterfly', 'munyeera', 'eggobe', 'ebiwojjolo', 'ebisoolisooli', 'namuginga', 'okugimusa', 'maize streak virus', 'ekirime', 'miceere', 'sikungula', 'lumonde', 'okukungula', 'cassava', 'ebirime', 'ebijanjaalo', 'weeding', 'garden', 'drought', 'leaves', 'insect', 'akatungulu', 'seed', 'pepper', 'matooke seedlings', 'harvesting', 'medicine', 'nursery bed', 'mucungwa', 'endwadde', 'pawpaw', 'enkota', 'ensiringanyi', 'kassooli', 'okufuuyira', 'caterpillars', 'ekijanjaalo', 'okukkoola', 'crop', 'okulima', 'endagala', 'kaamulali', 'ennima', 'omuceere', 'micungwa', 'ebisaanyi', 'plant', 'eddagala', 'ennimiro', 'amakoola', 'ebiwuka', 'ekigimusa', 'bibala', 'beans', 'nnimiro', 'ebinyebwa', 'passion fruit', 'Spinach', 'okuzifuuyira', 'ekirwadde', 'nakavundira', 'nfukir

# Modelling

For each class a GMMHMM is trained. 

Later for each testing data, we find the max score from fitted models.

In [135]:
def train_GMMHMM(dataset):
    GMMHMM_Models = {}
    states_num = 2#5
    GMM_mix_num = 3
    tmp_p = 1.0#1.0/(states_num-2)
    transmatPrior = np.array([[tmp_p, tmp_p, tmp_p, 0 ,0], \
                               [0, tmp_p, tmp_p, tmp_p , 0], \
                               [0, 0, tmp_p, tmp_p,tmp_p], \
                               [0, 0, 0, 0.5, 0.5], \
                               [0, 0, 0, 0, 1]],dtype=np.float)


    startprobPrior = np.array([0.5, 0.5, 0, 0, 0],dtype=np.float)

    for label in dataset.keys():
        print(label,len(dataset[label]))
        #model = hmm.GMMHMM(n_components=states_num, n_mix=GMM_mix_num, \
        #                   transmat_prior=transmatPrior, startprob_prior=startprobPrior, \
        #                   covariance_type='diag', n_iter=10)
        model = hmm.GMMHMM(n_components=1)
        #list of mfcc features for this label
        trainData = dataset[label]
        #mfcc length of each data point
        length = np.zeros([len(trainData), ], dtype=np.int)
        for m in range(len(trainData)):
            length[m] = trainData[m].shape[0]
        #convert list of 2d-arrays to array of 2D arrays
        trainData = np.vstack(trainData)
        model.fit(trainData, lengths=length)  # get optimal parameters
        GMMHMM_Models[label] = model
    return GMMHMM_Models

In [138]:
hmmModels = train_GMMHMM(featuredictrain)

akawuka 23
banana 3
obulwadde 21
nnyaanya 16
pampu 10
obutunda 16
plantation 6
ensujju 16
okulimibwa 11
mpeke 13
okusaasaana 23
ebigimusa 24
ekikolo 10
farm 5
kisaanyi 20
kikajjo 14
ekisaanyi 23
ndwadde 24
omusiri 10
butterfly 2
munyeera 23
eggobe 15
ebiwojjolo 21
ebisoolisooli 17
namuginga 23
okugimusa 24
maize streak virus 3
ekirime 12
miceere 14
sikungula 12
lumonde 16
okukungula 12
cassava 2
ebirime 10
ebijanjaalo 16
weeding 6
garden 5
drought 4
leaves 5
insect 4
akatungulu 14
seed 5
pepper 3
matooke seedlings 2
harvesting 5
medicine 3
nursery bed 5
mucungwa 15
endwadde 24
pawpaw 3
enkota 15
ensiringanyi 24
kassooli 20
okufuuyira 24
caterpillars 3
ekijanjaalo 14
okukkoola 11
crop 6
okulima 11
endagala 11
kaamulali 16
ennima 11
omuceere 15
micungwa 17
ebisaanyi 23
plant 5
eddagala 24
ennimiro 10
amakoola 12
ebiwuka 25
ekigimusa 19
bibala 15
beans 2
nnimiro 11
ebinyebwa 16
passion fruit 2
Spinach 3
okuzifuuyira 21
ekirwadde 19
nakavundira 21
nfukirira 12
onion 2
ddagala 24
muwogo 16


In [ ]:
hmmModels['emicungwa'].monitor_

ConvergenceMonitor(
    history=[-144306.97453369544, -144306.97453369544],
    iter=2,
    n_iter=10,
    tol=0.01,
    verbose=False,
)

In [140]:
score_cnt = 0
for label in featuredictest.keys():
    #get all data points of this label
    feature = featuredictest[label]
    scoreList = {}
    #repeat for each datapoint of testdata
    for index,testdata in enumerate(feature):
        #get score of feature for each model fitted to each label. max score gives predicted label
        for model_label in hmmModels.keys():
          #print(model_label)
          model = hmmModels[model_label]
          score = model.score(feature[index])
          scoreList[model_label] = score
        predict = max(scoreList, key=scoreList.get)
        print("Test on true label ", label, ": predict result label is ", predict)
        if predict == label:
          score_cnt+=1

Test on true label  akawuka : predict result label is  greens
Test on true label  akawuka : predict result label is  ebbugga
Test on true label  akawuka : predict result label is  afukirira
Test on true label  akawuka : predict result label is  farm
Test on true label  akawuka : predict result label is  omuyembe
Test on true label  akawuka : predict result label is  farm
Test on true label  banana : predict result label is  maize streak virus
Test on true label  obulwadde : predict result label is  akasaanyi
Test on true label  obulwadde : predict result label is  ebinyebwa
Test on true label  obulwadde : predict result label is  farm
Test on true label  obulwadde : predict result label is  ebbugga
Test on true label  obulwadde : predict result label is  farm
Test on true label  obulwadde : predict result label is  beans
Test on true label  nnyaanya : predict result label is  matooke
Test on true label  nnyaanya : predict result label is  ebikongoliro
Test on true label  nnyaanya : pre

In [141]:
testlen = 0
for label in featuredictest.keys():
  testlen += len(featuredictest[label])

In [142]:
print("Final recognition rate is %.2f"%(100.0*score_cnt/testlen), "%")

Final recognition rate is 2.03 %


<class 'int'>
